# PYPROTOLINC Demo

This notebook demonstrates how the different concepts within `pyprotolinc` can be used explaining various
customization options.

In [1]:
# we will need a bunch of imports for the below
from enum import unique

import numpy as np
import pandas as pd

from pyprotolinc.models.state_models import show_state_models, AbstractStateModel, state_model_by_name
import pyprotolinc._actuarial as actuarial
from pyprotolinc.riskfactors.risk_factors import get_risk_factor_names, Gender, SmokerStatus
from pyprotolinc.assumptions.iohelpers import AssumptionsLoaderFromConfig
from pyprotolinc.assumptions.providers import AssumptionType
# from pyprotolinc.assumptions.providers import AssumptionSetWrapper

## 1. State Models
As a first step a states model needs to be selected. We can choose between the built in states models or create a new one.

In [2]:
state_models = show_state_models()
state_models

{'AnnuityRunoffStates': <enum 'AnnuityRunoffStates'>,
 'MortalityStates': <enum 'MortalityStates'>,
 'MultiStateDisabilityStates': <enum 'MultiStateDisabilityStates'>}

We can look at the description of each of the built in models:

In [3]:
for state_model_name, state_model_class in state_models.items():
    print(state_model_class.describe())
    print()

AnnuityRunoffStates:  A state model consisting of two states:
        - DIS1 (=0) representing the annuity phase
        - DEATH (=1)
    

MortalityStates:  A state model with four states that can be used to model simple mortality term/perm products.
        - ACTIVE = 0
        - DEATH = 1
        - LAPSED = 2
        - MATURED = 3
    

MultiStateDisabilityStates:  A state model for a disabiility product with two disabled states. 
        - ACTIVE = 0
        - DIS1 = 1
        - DIS2 = 2
        - DEATH = 3
        - LAPSED = 4
    



We can now decide for one of the built-in models or create a custom model.

In [4]:
# decide for an existing one
# selected_state_model = state_model_by_name('MortalityStates')

# or create a new one by inheriting from AbstractStateModel
@unique
class DeferredAnnuityStates(AbstractStateModel):
    """ A state model for a deferred annuities. 
        - ACTIVE = 0
        - ANNUITANT = 1
        - LAPSED = 2
        - DEAD = 3
    """
    ACTIVE = 0
    ANNUITANT = 1  # 6
    LAPSED = 2
    DEAD = 3

# the new state model is now registered in pyprotolinc
show_state_models()

{'AnnuityRunoffStates': <enum 'AnnuityRunoffStates'>,
 'MortalityStates': <enum 'MortalityStates'>,
 'MultiStateDisabilityStates': <enum 'MultiStateDisabilityStates'>,
 'DeferredAnnuityStates': <enum 'DeferredAnnuityStates'>}

In [5]:
selected_state_model = state_model_by_name('DeferredAnnuityStates')

# print out the states
for state in selected_state_model:
    print(state)

DeferredAnnuityStates.ACTIVE
DeferredAnnuityStates.ANNUITANT
DeferredAnnuityStates.LAPSED
DeferredAnnuityStates.DEAD


### Mapping the States to the Standard Output Model
The next thing we need to specify is how the state should be mapped to the standatd output model
(if there is need for that at all)

In [6]:
# todo

## 2. Supplying Valuation Assumptions

We need to provide assumptions for the various state transitions in the model. In `pyprotolinc` this
is done by creating an object of type `AssumptionSet`. AssupmtionSets ar composed of `AssumptionProvides` and the the selection depends on `RiskFactors`.

### Risk Factors

Currently `pyprotolinc`supports five risk factors.

In [7]:
from pyprotolinc.riskfactors.risk_factors import _C_RISK_FACTORS
_C_RISK_FACTORS

{'Age': <CRiskFactors.Age: 0>,
 'Gender': <CRiskFactors.Gender: 1>,
 'CalendarYear': <CRiskFactors.CalendarYear: 2>,
 'SmokerStatus': <CRiskFactors.SmokerStatus: 3>,
 'YearsDisabledIfDisabledAtStart': <CRiskFactors.YearsDisabledIfDisabledAtStart: 4>}

In [8]:
# to better understand how the risk factors are encoded we can look at the values, e.g
[(g.name, g.value) for g in Gender]

[('M', 0), ('F', 1)]

In [9]:
[(g.name, g.value) for g in SmokerStatus]

[('S', 0), ('N', 1), ('A', 2), ('U', 3)]

### Building an Assumption Set

In [10]:
# the assumption set must have the dimension of the state models
states_dimension = len(selected_state_model)
acs = actuarial.AssumptionSet(states_dimension);

In [11]:
# to query the assumption set we need to provide values for each risk factor

# create a dummy realization of the risk factors
risk_factors = np.zeros(5, dtype=np.int32)
risk_factors[0] = 32              # age
risk_factors[1] = Gender.F        # gender
risk_factors[2] = 2023            # CalendarYear
risk_factors[3] = SmokerStatus.N  # SmokerStatus
risk_factors[4] = -1              # YearsDisabledIfDisabledAtStart

risk_factors

array([  32,    1, 2023,    1,   -1])

In [12]:
# at this stage we can already query
acs.get_single_rateset(risk_factors).reshape((states_dimension, states_dimension))

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

All possible state transitions have a (independent) transition probabilities of zero.

### Providers

Assumption Providers are multidimensional lookup tables that specify the transition probabilities for a single state transition depending on the risk factors.

In [13]:
# to illustrate that we specify a provider programmatically that depends on Gender and Age

vals2D = np.array([
       [0.1, 0.2, 0.3, 0.4],  # specifies Gender M and ages 0, 1, 2, 3
       [1.1, 1.2, 1.3, 1.4]   # specifies Gender M and ages 0, 1, 2, 3
], dtype=np.float64)

offsets = np.zeros(2, dtype=np.int32)
provider_test = actuarial.StandardRateProvider([actuarial.CRiskFactors.Gender, actuarial.CRiskFactors.Age], vals2D, offsets)

In [14]:
# we can query the provider for gender F and age 2
provider_test.get_rate((Gender.F, 2))

1.3

In [15]:
# we create a dummy provider programmatically for all ages
vals2d_long = (np.arange(120 * 2)).reshape((2, 120)) / 1000.0
vals2d_long

array([[0.   , 0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008,
        0.009, 0.01 , 0.011, 0.012, 0.013, 0.014, 0.015, 0.016, 0.017,
        0.018, 0.019, 0.02 , 0.021, 0.022, 0.023, 0.024, 0.025, 0.026,
        0.027, 0.028, 0.029, 0.03 , 0.031, 0.032, 0.033, 0.034, 0.035,
        0.036, 0.037, 0.038, 0.039, 0.04 , 0.041, 0.042, 0.043, 0.044,
        0.045, 0.046, 0.047, 0.048, 0.049, 0.05 , 0.051, 0.052, 0.053,
        0.054, 0.055, 0.056, 0.057, 0.058, 0.059, 0.06 , 0.061, 0.062,
        0.063, 0.064, 0.065, 0.066, 0.067, 0.068, 0.069, 0.07 , 0.071,
        0.072, 0.073, 0.074, 0.075, 0.076, 0.077, 0.078, 0.079, 0.08 ,
        0.081, 0.082, 0.083, 0.084, 0.085, 0.086, 0.087, 0.088, 0.089,
        0.09 , 0.091, 0.092, 0.093, 0.094, 0.095, 0.096, 0.097, 0.098,
        0.099, 0.1  , 0.101, 0.102, 0.103, 0.104, 0.105, 0.106, 0.107,
        0.108, 0.109, 0.11 , 0.111, 0.112, 0.113, 0.114, 0.115, 0.116,
        0.117, 0.118, 0.119],
       [0.12 , 0.121, 0.122, 0.123, 0.124, 0.12

In [16]:
provider = actuarial.StandardRateProvider([actuarial.CRiskFactors.Gender, actuarial.CRiskFactors.Age], vals2d_long, offsets)
provider.get_rate([Gender.F, 9])

0.129

In [17]:
# now we can add this provider to the assumption set for the state-transition 1 -> 0
acs.add_provider_std(1, 0, provider)

Now that we have added a provider we can query the assumption set again:

In [18]:
# query for the rates associated with the risk factors specified above
acs.get_single_rateset(risk_factors).reshape((states_dimension, states_dimension))

array([[0.   , 0.   , 0.   , 0.   ],
       [0.152, 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   ]])

The number (!=0) we see is in the second row (corresponding with STATE#1) and gives us the
probability to transition to STATE#0 (first column).

### Building AssumptionSets from Files

Instead manually bootstrapping the required object there is a more convenient way for simpler cases to **create 
assumptions sets from configruration files and spreadsheets**.

In [19]:
assumption_config_loader = AssumptionsLoaderFromConfig(r"..\di_assumptions.yml", states_dimension)
assumption_config_loader.assumptions_spec

{'be': [[0,
   3,
   ['FileTable', 'base_assumptions_annuity.xlsx', 'MORTALITY (0->3)']],
  [0, 2, ['FileTable', 'base_assumptions_annuity.xlsx', 'LAPSE (0->2)']],
  [1, 3, ['FileTable', 'base_assumptions_annuity.xlsx', 'DIS_DEATH (1->3)']]],
 'res': [[0,
   3,
   ['FileTable', 'base_assumptions_annuity.xlsx', 'MORTALITY (0->3)']],
  [0, 2, ['FileTable', 'base_assumptions_annuity.xlsx', 'LAPSE (0->2)']],
  [1, 3, ['FileTable', 'base_assumptions_annuity.xlsx', 'DIS_DEATH (1->3)']]]}

In [20]:
assumption_set_wrapper = assumption_config_loader.load()

In [21]:
# some possible checks
# assumption_set_wrapper.be_transitions
# assumption_set_wrapper.be_transitions[0][2].get_values()
# assumption_set_wrapper.build_rates_provides_matrix(AssumptionType.BE)

In [22]:
# now we construct an assumption set from the wrapper and query again:
acs2 = assumption_set_wrapper.build_assumption_set(AssumptionType.BE)
acs2.get_single_rateset(risk_factors).reshape((states_dimension, states_dimension))

array([[0.      , 0.      , 0.049   , 0.000725],
       [0.      , 0.      , 0.      , 0.06    ],
       [0.      , 0.      , 0.      , 0.      ],
       [0.      , 0.      , 0.      , 0.      ]])

In [23]:
# risk_factors

## 3. Record Data

The next input we need is the portfolio data, i.e. file containing seriatim records. The most convenient way is to import a portfolio from an Excel file using a ``pyprotolinc.portfolio.PortfolioLoader`` object.

In [24]:
import pyprotolinc.portfolio as ptf

portfolio_path = "../portfolio_small3.xlsx"
ptf_loader = ptf.PortfolioLoader(portfolio_path)

Portfolio files have to look like this (where the order of the columns does not matter):

In [25]:
df_portfolio = pd.read_excel(portfolio_path)
df_portfolio

,DATE_PORTFOLIO,ID,DATE_OF_BIRTH,DATE_START_OF_COVER,SUM_INSURED,CURRENT_STATUS,SEX,PRODUCT,PRODUCT_PARAMETERS,SMOKERSTATUS,RESERVING_RATE,DATE_OF_DISABLEMENT
0,2021-12-31,1,1976-04-23,2022-01-01,1000,ACTIVE,m,DeferredAnnuity,NaN,U,0,NaN
1,2021-12-31,2,1969-03-03,2022-01-01,100,ACTIVE,f,DeferredAnnuity,NaN,U,0,NaN
2,2021-12-31,3,1970-09-13,2022-01-01,5000,ANNUITANT,f,DeferredAnnuity,NaN,U,0,NaN


The columns have the obvious meanings:

* DATE_PORTFOLIO the date at which the snapshot is valid, must be equal for all records.
* ID a unique indentifier
* DATE_OF_BIRTH
* DATE_START_OF_COVER
* SUM_INSURED sum insured (or yearly annuity amount)
* CURRENT_STATUS corresponds with the state in the state model the record is in at DATE_PORTFOLIO
* SEX gender (m/f)
* PRODUCT is a string that references a product
* PRODUCT_PARAMETERS additional product parameters
* SMOKERSTATUS
* RESERVING_RATE the interest rate that should be used for the reserve calculations
* DATE_OF_DISABLEMENT
    
When importing using the `loader` certain validations will be performed on the fly and to be able to validate the the status a corresponding state model class must be passed in to load the portfolio:

In [26]:
# load the portfolio object
portfolio = ptf_loader.load(selected_state_model)
portfolio

<Portfolio with 3 records and state model <enum 'DeferredAnnuityStates'>

Moreover, to save some time a cacheing of the portfolios is implemented when instatiating with a second path::
 
    ptf.PortfolioLoader(portfolio_path, cache_path)

Then pickled versions of the portfolios read will be stored under the path and searching for portfolios to load will be done there first.

### 4. Products

The final ingredient needed in the definition of the life insurance product. As already seen the product is specified
on the policy level in the portfolio file as a string.

In [27]:
##################
# TODO


### 5. Main Configuration Object

To start a run we have to combine all the relevant information in a single configuration object.

In [28]:
from importlib import reload

import pyprotolinc
reload(pyprotolinc)

<module 'pyprotolinc' from 'D:\\programming\\pyprotolinc\\src\\pyprotolinc\\__init__.py'>

Most of the values have sensible defaults but one must be provided: the name of the state model.

In [29]:
run_cfg = pyprotolinc.RunConfig(working_directory = ".", state_model_name=selected_state_model.__name__)

The object's ``__init__`` method has the following signature:
    
    def __init__(self,
                 state_model_name: str,
                 working_directory: Path = ".",
                 model_name: str = "GenericMultiState",
                 years_to_simulate: int = 120,
                 steps_per_month: int = 1,
                 portfolio_path: Optional[str] = None,
                 assumptions_path: Optional[str] = None,
                 outfile: str = "ncf_out_generic.csv",
                 portfolio_cache: Optional[str] = None,
                 profile_out_dir: Optional[str] = None,
                 portfolio_chunk_size: int = 20000,
                 use_multicore: bool = False,
                 kernel_engine: str = "PY",
                 max_age: int = 120
                 ) -> None:

It is also possible and often more convenient to load a configuration object from a file.

In [30]:
run_cfg2 = pyprotolinc.get_config_from_file("../config.yml")

### 6. Starting a Run

Runs can be triggered by two methods in the module ``pyprotolinc.main``: either ``project_cashflows`` or ``project_cashflows_cli``.

If using the latter all data must be delivered via the config file path or object. The former allows
to inject programmatic objects.

In [31]:
from pyprotolinc.main import project_cashflows

In [33]:
project_cashflows(run_cfg, df_portfolio, assumption_set_wrapper, export_to_file=False)

INFO - 2023-03-24 17:35:50,796 - pyprotolinc.main - Multistate run with config: {'working_directory': '.', 'model_name': 'GenericMultiState', 'years_to_simulate': 119, 'portfolio_path': None, 'assumptions_path': None, 'steps_per_month': 1, 'state_model_name': 'DeferredAnnuityStates', 'timestep_duration': 0.08333333333333333, 'outfile': 'ncf_out_generic.csv', 'portfolio_cache': None, 'profile_out_dir': None, 'portfolio_chunk_size': 20000, 'use_multicore': False, 'kernel_engine': 'PY', 'max_age': 119}
DEBUG - 2023-03-24 17:35:50,798 - pyprotolinc.portfolio - Initializing portfolio from dataframe
DEBUG - 2023-03-24 17:35:50,810 - pyprotolinc.portfolio - Splitting portfolio for product DEFERREDANNUITY.
DEBUG - 2023-03-24 17:35:50,813 - pyprotolinc.portfolio - Initializing portfolio from dataframe
DEBUG - 2023-03-24 17:35:50,826 - pyprotolinc.portfolio - Initializing portfolio from dataframe
DEBUG - 2023-03-24 17:35:50,836 - pyprotolinc.portfolio - Initializing portfolio from dataframe
INFO

KeyError: 'DEFERREDANNUITY'